In [8]:
import firebase_admin
from firebase_admin import credentials, firestore

# cred = credentials.Certificate("../recyclesg-firebase-adminsdk-serviceAccountKey.json")
# cred = credentials.Certificate("../serviceAccountKey.json")

# app = firebase_admin.initialize_app(
    # credential=cred,
# )
app = firebase_admin.initialize_app()

db = firestore.client()



# Showing all collections in the db

In [19]:
for coll in db.collections():
    print(coll.id)

recyclable
recyclerequests
search
test
users
view_search_results
wasteType


# Getting documents in `wasteType` collection

In [20]:
collection = db.collection("wasteType")

In [21]:
# Get all docs in a collection
docs = collection.get()
print(f"There are {len(docs)} documents in the collection")

docs[0].to_dict()

There are 430 documents in the collection


{'links': ['https://www.nea.gov.sg/our-services/waste-management/3r-programmes-and-resources/e-waste-management/where-to-recycle-e-waste'],
 'instructions': 'Can be recycled through: • Retailer 1-for-1 Take-back• Bulky item removal service by TCs for HDB residents• E-waste collection drive (Quarterly)• Doorstep collection (fees may apply) To find out where you can recycle your e-waste, visit here.*Please note that the small household appliances will be collected and recycled under recycling programmes organised under the National Voluntary Partnership for the Proper Management of Non-Regulated Used Household Electrical/Electronic Products.',
 'item': 'nintendo',
 'recyclable': True,
 'material': 'Others'}

In [14]:
# Get all docs in a collection
docs = db.collection("wasteType").stream()

items_db = []
for doc in docs:
    items_db.append(doc.to_dict())
len(items_db)



436

## Deleting documents from a collection

In [ ]:
# Function to delete a collection
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

# Usage
delete_collection(collection, batch_size=50)



# Querying a collection

In [16]:
# get a document in a collection by item name
# Note: Use of CollectionRef stream() is prefered to get()
from google.cloud.firestore_v1.base_query import FieldFilter

docs = (
    collection
    .where(filter=FieldFilter("item", "==", "eraser"))
    .stream()
)

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

hqfZ1bXb9OXnTydwsYPU => {'instructions': 'Donate if it is in good condition.', 'item': 'eraser', 'recyclable': False, 'material': 'Others', 'id': 'hqfZ1bXb9OXnTydwsYPU'}


In [31]:
# find by id
db.collection("wasteType").document("9r28RHwQgvvUfI7IC25M").get().to_dict()

{'instructions': 'Donate if it is in good condition.',
 'item': 'Eraser',
 'recyclable': False,
 'material': 'Others',
 'id': '668250ca-fcac-4ddb-985b-debea16b97d1'}

# Seeding data into database

In [18]:
import json
# Start a batch
batch = db.batch()

# Data to add
with open('../data/cleaned_data_final.json', 'r') as file:
    data = json.load(file)

# Add documents to batch
for doc_data in data:
    doc_data['item'] = doc_data['item'].lower()
    doc_ref = collection.document()  # Create a new document reference
    batch.set(doc_ref, doc_data)  # Add to batch

# Commit the batch
batch.commit()

[update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_

In [20]:
# Note: Use of CollectionRef stream() is prefered to get()
docs = (
    collection
    .where(filter=FieldFilter("item", "in", ["eraser", "vegetables", "pencil box"]))
    .stream()
)

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

0e1kC4AfnFSkeTyUR8i8 => {'instructions': 'should be disposed of as general waste.', 'item': 'vegetables', 'recyclable': False, 'material': 'Others', 'id': '47850322-8c61-4f32-ab37-8bb1a24a98fd'}
AUTVwYEMliQuyeRP2yjY => {'instructions': 'Donate if it is in good condition.', 'item': 'eraser', 'recyclable': False, 'material': 'Others', 'id': '0937acbe-5571-4919-9d9c-897cd236da02'}


In [34]:
s = 's'
s.strip()

's'